In [1]:
!pip install dotenv
!pip install qdrant-client
!pip install --upgrade google-genai
!pip install pdf2image
!pip install pypdf
!pip install pillow
!apt-get install poppler-utils
! pip install -qU langchain-qdrant
! pip install langchain-core
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.7/159.7 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.10.0
    Uninstalling google-genai-1.10.0:
      Successfully uninstalled google-genai-1.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 0s (898 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126332 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
import google.generativeai as genai
from google.genai.types import HttpOptions
from pdf2image import convert_from_path
from PIL import Image
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from uuid import uuid4
from langchain_openai import OpenAIEmbeddings
import glob
from collections import defaultdict

Mounted at /content/drive


In [3]:

#### ENVIRONMENT VARIABLES######
QDRANT_KEY='xxxx'
GEMINI_KEY='xxxx'
QDRANT_URL='https://02d13de5-fafe-4288-822a-5aaca32e31a9.europe-west3-0.gcp.cloud.qdrant.io'
OPEN_AI_KEY='xxxxx'

##RETRIEVER CONFIGURATIONS###
model_name = "sentence-transformers/all-MiniLM-L6-v2"
hf = OpenAIEmbeddings(
    api_key=OPEN_AI_KEY,
    model="text-embedding-3-small"
)
# Configure client all operation involving the db should be done through this object
qdrant_vector_store=QdrantVectorStore.from_existing_collection( api_key=QDRANT_KEY ,
                                                               embedding=hf,url=QDRANT_URL,
                                                                collection_name='xxxx',
                                                                prefer_grpc=True)


In [5]:
# Configure LLM model for pre-processing tasks
genai.configure(api_key=GEMINI_KEY)
model=genai.GenerativeModel('gemini-2.0-flash-001')

In [11]:
###Logica de programa###
'''
Para convertir los archivos pdf a chunks jerarquicos se realizo el siguiente procedimiento:
- convertimos individualmente cada unas de las paginas del archivo pdf a imagen  usando el metodo convert_to_images el resultado final se almacena en una carpeta por ejemplo 'PDF_IMAGES'
- sobre las imagenes almacenadas usamos el LLM para extraer el contenido y generar archivos markdown usando el metodo ocr_with_gemini los archivos generados son almacenados en otra carpeta por ejemplo 'MARKDOWN_FILES'
- los chunks son generados usando el mismo LLM usando la funcion create_chunks
- para subir los chunks creados es necesario utilizar la funcion create_documents y upload_to_qdrant la primera funcion crea las abstracciones necesarias para realizar busquedas usando el framework langchain
   la segunda funcion carga los datos usando el cliente de qdrant.
'''

## Procesamiento de imagenes
def convert_to_images(pdf_path):
  input_dir='/content/drive/MyDrive/Riesgo Financiero/IAS/'
  output_dir='/content/drive/MyDrive/PDF_IMAGES/'
  inppdf_path=input_dir+pdf_path
  images= convert_from_path(inppdf_path)
  image_paths=[]
  for i,image in enumerate(images):
    image_path=output_dir+pdf_path.split('.')[0]+f'_page_{i+1}.jpg'
    image.save(image_path,'JPEG')
    image_paths.append(image_path)
  return image_paths


# Generacion de markdown
def ocr_with_gemini(image_paths,model):
  images=[(path,Image.open(path))for path in image_paths]
  prompt=f"""

Extract the contents of the provided iamge into a Markdown format.  Pay close attention to preserving the document's structure, including headings, lists, and paragraphs. For tables, implement the following rules:

1. **Ideal Extraction:** If the table's data can be accurately extracted, represent it in a Markdown table format.

2. **Sanitary Extraction (Data Extraction Failure):** If the table data cannot be reliably extracted, provide a brief summary of the table's content in a Markdown paragraph. Include the number of rows and columns and a general description of the table's subject matter.  For example:  "This table (5 rows x 3 columns) appears to summarize key financial ratios for Q3 2023."

3. **Image Handling:** If a table is represented as an image, describe it in a similar manner to rule 2, referencing the image if possible.

Ensure that all extracted content maintains semantic meaning. Use Markdown formatting consistently.

"""
  summary=""

  for path,image in images:
      try:
        response=model.generate_content([prompt,image])
        summary+=response.text+'####End####'
      except:
        print(f'Error procesando la imagen {path}')
        continue
  return summary

#creacion de chunks
def create_chunks(path_file,chunk_size):
  #Retrieve and open file
  with open(path_file,'r') as file:
    markdown_content=file.read()

  markdown_content=markdown_content.split('####End####')
  markdown_content= [ page_content+str(page_number).replace('@','') for page_number,page_content in enumerate(markdown_content)]

  prompt_chunk=f"""
  Based on the content of a file create text chunks of max {chunk_size} words
  The file strucutre is a markdown hence use the following guidelines to handdle different types of content:
    Preserve hierarchy: Use the heading levels to determine the chunk boundaries. For example:
        - A *bold title* or ## begins a new chunk.
        - A ### heading begins a sub-sub-section and so on.
    Chunking rule: Each chunk should include:
    Its full heading path as metadata (e.g., Chapter 1 > Section 1.1 > Subsection A).
    All relevant content under that heading until the next heading of the same or higher level.
    Maximum size per chunk (optional): If any section exceeds {chunk_size} words or tokens, split it further by semantic or paragraph boundaries, and maintain the heading path in each sub-chunk.

  only provide the chunk information in the ouput dont icnlude any other extra information regarding your reasoning use the following strucuture in the for each chunk dont use json in your answer :
  <chunk_information>
  <heading_path>:most recent bold title or heading identified </heading_path>
  <content>:The chunk content</content>
  <page_numer>: use the number provided as input for this value</page_numer>
  <name_file>: name of the file</name_file>

  </chunk_information>
  """
  chunk_list=[]
  for markdown_chunk in markdown_content:
    try:
      response=model.generate_content([prompt_chunk,markdown_chunk])
      chunk_list.append(response.text)
    except:
      print(f'Error procesando el chunk {markdown_chunk}')
      continue

  return chunk_list

### Subir chunks a Qdrant###
def create_documents(text_blob):
  import re
  chunks=re.findall(r"<chunk_information>(.*?)</chunk_information>", text_blob,re.DOTALL)
  documents=[]
  for chunk in chunks:
    try:
      heading_path=re.search(r"<heading_path>(.*?)</heading_path>", chunk).group(1) if re.search(r"<heading_path>(.*?)</heading_path>", chunk) else 'invalid_heading'
      page_number=re.search(r"<page_numer>(.*?)</page_numer>", chunk).group(1)if re.search(r"<page_numer>(.*?)</page_numer>", chunk) else '99999'
      name_file=re.search(r"<name_file>(.*?)</name_file>", chunk).group(1) if re.search(r"<name_file>(.*?)</name_file>", chunk) else 'invalid_name_file'
      content=re.search(r"<content>(.*?)</content>", chunk,re.DOTALL).group(1) if re.search(r"<content>(.*?)</content>", chunk,re.DOTALL) else 'invalid_content'
      if heading_path!='invalid_heading' and page_number!='invalid_page_number' and name_file!='invalid_name_file' and content!='invalid_content':
            documents.append(Document(page_content=content,
                              metadata={'heading_path':heading_path,'page_number':page_number,'name_file':name_file})
            )
      if heading_path=='invalid_heading' or page_number=='invalid_page_number' or name_file=='invalid_name_file' or content=='invalid_content':
        dict_debuging={'heading_path':heading_path,'page_number':page_number,'name_file':name_file,'content':len(content)}
        print(f'Error procesando el chunk document built:{dict_debuging}')
        continue
    except Exception as e:
        continue;


  return documents
def upload_to_qdrant(document_list,db_client):
  uuids = [str(uuid4()) for _ in range(len(document_list))]
  db_client.add_documents(
      documents=document_list,
      ids=uuids)




SyntaxError: incomplete input (<ipython-input-11-7fbeb30ef692>, line 2)

In [42]:
import json
for  blob_text in glob.glob('/content/drive/MyDrive/MARKDOWN_FILES/*.txt'):
  try:
    with open(blob_text,'r') as f:
      blob_content=f.read()
    documents=create_documents(blob_content)
    json_documents=[document.json() for document in documents]
    with open(f'{blob_text.replace(".txt",".json")}','w') as f:
      json.dump(json_documents,f)
    upload_to_qdrant(documents,qdrant_vector_store)
  except Exception as e:
    raise e
    continue

<ipython-input-42-ef81947ccab8>:7: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json_documents=[document.json() for document in documents]


Error procesando el chunk document built:{'heading_path': 'NIIF 9 > Capítulo 5 Medición', 'page_number': '22', 'name_file': 'NIIF 9', 'content': 15}
Error procesando el chunk document built:{'heading_path': 'invalid_heading', 'page_number': ': 4 ', 'name_file': ': IAS 36', 'content': 2243}
Error procesando el chunk document built:{'heading_path': 'invalid_heading', 'page_number': ':20', 'name_file': ': Información a revelar', 'content': 2358}
Error procesando el chunk document built:{'heading_path': 'invalid_heading', 'page_number': '20', 'name_file': 'NIC 32', 'content': 1946}
Error procesando el chunk document built:{'heading_path': 'invalid_heading', 'page_number': '42', 'name_file': 'NIC 32', 'content': 1268}
